<a href="https://colab.research.google.com/github/SatyabrataBhattacharjee/PharmaRAG--A-Retrieval-Augmented-Generation-RAG-powered-medical-assistant-for-drug-related-queries./blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/gurgaon_properties_post_feature_selection_v2 (3).csv')

In [ ]:
df.isnull().sum()

,0
property_type,0
sector,0
price,0
bedRoom,0
bathroom,0
balcony,0
agePossession,0
built_up_area,0
servant room,0
store room,0


In [ ]:
df['balcony']=df['balcony'].apply(lambda x: '3' if x=='3+' else x)

In [ ]:
df

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3,Relatively New,1615.0,1.0,0.0,1.0,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,0.0,High,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,flat,sector 84,0.37,2.0,2.0,1,Relatively New,532.0,0.0,0.0,0.0,Medium,Mid Floor
3550,house,sector 109,6.00,5.0,5.0,3,Relatively New,6228.0,1.0,1.0,0.0,High,Low Floor
3551,flat,sector 2,0.60,1.0,1.0,1,Moderately Old,665.0,0.0,0.0,1.0,Medium,Mid Floor
3552,house,sector 43,15.50,5.0,6.0,3,Moderately Old,5490.0,1.0,1.0,0.0,Medium,Mid Floor


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

class PropertyRecommender:
    def __init__(self, df, regression_model_path):
        self.df = df.copy()
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Create natural language descriptions
        self.df['description'] = self.df.apply(self._row_to_sentence, axis=1)
        self.embeddings = self.model.encode(self.df['description'].tolist(), show_progress_bar=True)

        # Load regression pipeline
        self.regressor = joblib.load(regression_model_path)

        # Predict log(price) → actual price for all properties
        try:
            features_df = self.df[[
                'property_type', 'sector', 'bedRoom', 'bathroom', 'balcony', 'agePossession',
                'built_up_area', 'servant room', 'store room', 'furnishing_type',
                'luxury_category', 'floor_category'
            ]]
            log_preds = self.regressor.predict(features_df)
            self.df['predicted_price'] = np.expm1(log_preds)
        except Exception as e:
            print("Error predicting prices for dataset:", e)
            self.df['predicted_price'] = np.nan

    def _row_to_sentence(self, row):
        parts = []
        if pd.notnull(row.get('bedRoom')): parts.append(f"{int(row['bedRoom'])}BHK")
        if pd.notnull(row.get('property_type')): parts.append(f"{row['property_type']}")
        if pd.notnull(row.get('sector')): parts.append(f"in {row['sector']}")
        if pd.notnull(row.get('bathroom')): parts.append(f"with {row['bathroom']} bathrooms")
        if pd.notnull(row.get('balcony')): parts.append(f"and {row['balcony']} balconies")
        if pd.notnull(row.get('furnishing_type')): parts.append(f"{row['furnishing_type']} furnishing")
        if pd.notnull(row.get('built_up_area')): parts.append(f"{int(row['built_up_area'])} sqft area")
        if pd.notnull(row.get('agePossession')): parts.append(f"{row['agePossession']} condition")
        if pd.notnull(row.get('floor_category')): parts.append(f"{row['floor_category']} floor")
        if pd.notnull(row.get('luxury_category')): parts.append(f"{row['luxury_category']} luxury")
        return ' '.join(parts)

    def _input_to_sentence(self, user_input):
        parts = []
        if 'bedRoom' in user_input: parts.append(f"{int(user_input['bedRoom'])}BHK")
        if 'property_type' in user_input: parts.append(f"{user_input['property_type']}")
        if 'sector' in user_input: parts.append(f"in {user_input['sector']}")
        if 'bathroom' in user_input: parts.append(f"with {user_input['bathroom']} bathrooms")
        if 'balcony' in user_input: parts.append(f"and {user_input['balcony']} balconies")
        if 'furnishing_type' in user_input: parts.append(f"{user_input['furnishing_type']} furnishing")
        if 'built_up_area' in user_input: parts.append(f"{int(user_input['built_up_area'])} sqft area")
        if 'agePossession' in user_input: parts.append(f"{user_input['agePossession']} condition")
        if 'floor_category' in user_input: parts.append(f"{user_input['floor_category']} floor")
        if 'luxury_category' in user_input: parts.append(f"{user_input['luxury_category']} luxury")
        return ' '.join(parts)

    def recommend(self, user_input, top_n=5):
        # Predict log(price) and convert to Cr
        input_df = pd.DataFrame([user_input])
        try:
            log_price = self.regressor.predict(input_df)[0]
            predicted_price = np.expm1(log_price)
        except Exception as e:
            print("Error predicting user price:", e)
            predicted_price = None

        # Get semantic similarity
        query_sentence = self._input_to_sentence(user_input)
        query_embedding = self.model.encode([query_sentence])
        sims = cosine_similarity(query_embedding, self.embeddings)[0]
        top_indices = sims.argsort()[-top_n:][::-1]

        # Extract and format results
        results = self.df.iloc[top_indices].copy()
        results['user_predicted_price'] = predicted_price

        # Format currency columns to ₹X.XX Cr
        for col in ['price', 'predicted_price', 'user_predicted_price']:
            results[col] = results[col].apply(lambda x: f"₹{x:.2f} Cr" if pd.notnull(x) else "N/A")

        # Reorder columns for clean display
        columns_order = [
            'sector', 'property_type', 'bedRoom', 'bathroom', 'balcony', 'built_up_area',
            'furnishing_type', 'floor_category', 'luxury_category',
            'price', 'predicted_price', 'user_predicted_price'
        ]
        return results[columns_order], f"₹{predicted_price:.2f} Cr" if predicted_price else "N/A"





In [ ]:
import pandas as pd

# Load data


# Initialize recommender
recommender = PropertyRecommender(df, "/content/pipeline (1).pkl")

# User query
user_input = {
    'property_type': 'flat',
    'sector': 'sector 102',
    'bedRoom': 4,
    'bathroom': 3,
    'balcony': '3+',
    'agePossession': 'New Property',
    'built_up_area': 2750,
    'servant room': 0,
    'store room': 0,
    'furnishing_type': 'unfurnished',
    'luxury_category': 'Low',
    'floor_category': 'Low Floor'
}

# Get top 5 similar properties
recommendations = recommender.recommend(user_input, top_n=5)
recommendations


Batches:   0%|          | 0/112 [00:00<?, ?it/s]

Error predicting prices for dataset: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''


(          sector property_type  bedRoom  bathroom balcony  built_up_area  \
 119   sector 102          flat      4.0       4.0       3         4630.0   
 1786  sector 102          flat      4.0       5.0       3         2600.0   
 3278  sector 103          flat      4.0       5.0       3         2416.0   
 1741  sector 102          flat      4.0       4.0       3         3178.0   
 222   sector 110          flat      4.0       4.0       3         3350.0   
 
       furnishing_type floor_category luxury_category     price  \
 119               1.0     High Floor          Medium  ₹4.55 Cr   
 1786              1.0      Mid Floor            High  ₹2.50 Cr   
 3278              0.0     High Floor             Low  ₹2.00 Cr   
 1741              1.0      Mid Floor          Medium  ₹3.00 Cr   
 222               0.0     High Floor          Medium  ₹4.00 Cr   
 
      predicted_price user_predicted_price  
 119              N/A             ₹2.11 Cr  
 1786             N/A             ₹2.11 Cr

In [ ]:
top_matches, user_price = recommender.recommend(user_input, top_n=5)

print(f"\n🔮 Predicted price for your input: {user_price}")
print("\n🏠 Top matching properties:\n")
print(top_matches.to_string(index=False))



🔮 Predicted price for your input: ₹3.22 Cr

🏠 Top matching properties:

    sector property_type  bedRoom  bathroom balcony  built_up_area  furnishing_type floor_category luxury_category    price predicted_price user_predicted_price
sector 103         house      4.0       4.0       3         3222.0              1.0      Low Floor          Medium ₹4.35 Cr             N/A             ₹3.22 Cr
 sector 91         house      4.0       4.0       3         2430.0              1.0      Low Floor             Low ₹3.60 Cr             N/A             ₹3.22 Cr
sector 112         house      4.0       4.0       3         6350.0              1.0      Low Floor          Medium ₹7.46 Cr             N/A             ₹3.22 Cr
sector 112         house      4.0       6.0       3         6382.0              1.0      Low Floor          Medium ₹7.99 Cr             N/A             ₹3.22 Cr
sector 108         house      4.0       4.0       3         3000.0              0.0      Low Floor             Low ₹1.70 C